In [45]:
import numpy as np
import cv2
import sys
import time
from collections import deque

In [49]:
# Taking image of background
cap = cv2.VideoCapture(0)

time.sleep(3)
background=0

for i in range(30):
    ret, background = cap.read()

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# VideoWriter object
out = cv2.VideoWriter('output1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 24, (frame_width,frame_height))    

background = cv2.flip(background, 1)
cap.release()
cv2.destroyAllWindows()

In [50]:
# lower and upper boundaries of the "object_color"
color_Lower = (110, 85, 5)   
color_Upper = (130,255,255)       
pts = deque(maxlen=240)

In [55]:
cap = cv2.VideoCapture(0)

while(True):
    # Frame-by-frame
    ret, img = cap.read()
    
    # Flip the image
    img = cv2.flip(img, 1)
    
    # Blur and BGR to HSV 
    blurred = cv2.GaussianBlur(img, (11, 11), 0)
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    
    # Range for lower red (color of cloak)
    lower_red = np.array([0,150,70])
    upper_red = np.array([20,255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    # Range for upper range
    lower_red = np.array([160,150,70])
    upper_red = np.array([180,255,255])
    mask2 = cv2.inRange(hsv,lower_red,upper_red)
    
    # Final mask to detect red color
    mask1 = mask1 + mask2
    
    mask2 = cv2.bitwise_not(mask1)
    mask2 = cv2.erode(mask2, None, iterations=5)
    mask2 = cv2.dilate(mask2, None, iterations=5)
    
    #Segmenting the cloth out of the frame using bitwise_and with the inverted mask
    res1 = cv2.bitwise_and(img, img, mask=mask2)
    # creating image showing static background frame pixels only for the masked region
    res2 = cv2.bitwise_and(background, background, mask = mask1)

    # Output with invisibility-cloak
    final_output = cv2.addWeighted(res1,1,res2,1,0)
    
    ### Object-tracking, by color
    
    mask = cv2.inRange(hsv, color_Lower, color_Upper)
    mask = cv2.erode(mask, None, iterations=5)
    mask = cv2.dilate(mask, None, iterations=5)
    
    # Find contours in the mask and initialize the current(x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    center = None
    
    if len(cnts) > 0:
        # Find the largest contour in the mask, then use it to compute the minimum enclosing circle and centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        # Only proceed if the radius meets a minimum size
        if radius > 50:
            # Draw the circle and centroid on the frame,then update the list of tracked points
            cv2.circle(final_output, (int(x), int(y)), int(radius), (255, 0, 0), 2)
            cv2.circle(final_output, center, 5, (255, 0, 0), -1)
    pts.appendleft(center)
    
    # Loop over the set of tracked points
    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue
        cv2.line(final_output, pts[i - 1], pts[i], (255, 0, 0), 3)
    
    # Final Output
    out.write(final_output)
    cv2.imshow("Final", final_output)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [53]:
cap.release()
cv2.destroyAllWindows()

------------------------------------------------------------